# Nematic Transition for Heisenberg Model

We will determine the self-consistent bond equation $K_{\mathbf{q}}^{\mathrm{eff}}=K_{\mathbf{q}}+\int_{\mathbf{p}}\frac{T^2D_\mathbf{p}}{K_{\mathbf{p+q}}^{\mathrm{eff}}}$ with $D_{\mathbf{p}}=\frac{2}{NT^2}\left(\int_{\mathbf{k}}\frac{1}{K_{\mathbf{k}}^{\mathrm{eff}}}\frac{1}{K_{\mathbf{k+p}}^{\mathrm{eff}}}\right)^{-1}$, and check for spontaneous symmetry breaking of $C_{4V}$.

In [103]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Set values for size, spin and J.
m = 10 #size of 2d-lattice (mxm)
NS = 3.0 #number of spincomponents
J1 = 1.0
J2 = 0.0
J3 = 0.0

DeltaList = np.fromfunction(lambda qx,qy: 10**-9*(10**qy),(1,11))[0]#np.linspace(5*10**(-4),0.1*10**(0),100) #give list of Delta (changes temp.)

maxNsteps = 10 #max itt. each step
convCrit = 10**(-15) #decides when it has converged
checkNSteps = 10 #check convergence every n-step

da = 0.1 #adds symmetrybreaking term to K.
dq = 2.0*np.pi/float(m) #constant for later
nTemp = len(DeltaList) #number of temp.
nStepsNow = 0
stepsInN = 0

Setting up functions and tables:

In [104]:
#making JFTtab:
def J (qx, qy):
    return (-J1*(np.cos(qx) + np.cos(qy)) + 2*J2*np.cos(qx)
            *np.cos(qy) + J3*(np.cos(2.0*qx) + np.cos(2.0*qy)))

JFT=np.fromfunction(lambda qx,qy: J(dq*qx,dq*qy), (m,m)) #qx vertical, qy horisontal 


#making dK tab for symmetrybreaking
def dKfunc (qx,qy):
    return np.cos(qx)*np.sin(np.sqrt(2.0)*qy)+np.cos(np.sqrt(3.0)*(qx+0.4*qy))

dKtab = np.fromfunction(lambda qx,qy: dKfunc(dq*qx,dq*qy), (m,m)) 


#making tables to check for symmetry brekaing along axis or diagonal
def funcAxis(qx,qy):
    return np.cos(qx)-np.cos(qy)
def funcDia(qx,qy):
    return np.cos(qx+qy)-np.cos(qx-qy)

symTabAxis = np.fromfunction(lambda qx,qy: funcAxis(dq*qx,dq*qy), (m,m)).flatten()
symTabDia = np.fromfunction(lambda qx,qy: funcDia(dq*qx,dq*qy), (m,m)).flatten()


#calculates temp, once K and self-energy is known
def Temp(K1):
    return ((2.0/NS)*m*m/(np.sum(np.reciprocal(K1))))

Making tables for saving results:

In [105]:
sigma = pd.DataFrame(np.zeros((maxNsteps,3)), \
                         columns=['Delta', 'sigmaAxis', 'sigmaDia'])

sigmaTemp = pd.DataFrame(np.zeros((nTemp,4)), \
                         columns=['Delta','Temperature', 'sigmaAxis', 'sigmaDia'])

Begining iteration:

In [106]:
for temp in range(0,nTemp):
    #iterating through all values of Delta
    Delta=DeltaList[temp]
    
    K0 = JFT - np.min(JFT) + Delta #making sure K0 is positive
    K1 = K0 + da*dKtab -np.min(K0 + da*dKtab)+Delta #adding symmetry breaking term to K0
    
    sigma.iloc[0] = np.array([[Delta,np.dot(symTabAxis,K1.flatten())/(m*m),
          np.dot(symTabDia,K1.flatten())/(m*m)]]) #calculating sigmas starting value

    for i in range(1,maxNsteps): #iterating while checking for convergense
        K1inv = np.reciprocal(K1)
        
        #calculating D with convolution
        invK1Real = np.fft.ifft2(K1inv)
        invK1RealConj = np.conj(invK1Real)
        invK1RealMult = np.multiply(invK1Real, invK1RealConj)
        invK1Conv = np.real(np.fft.fft2(invK1RealMult))#get D1 real,complex part very small.
        D1 = (2.0/NS)*np.reciprocal(invK1Conv) 

        #adding self-energy with convolution
        D1Real = np.fft.ifft2(D1)
        D1K1RealMult =  np.multiply(D1Real, invK1RealConj)
        D1K1Conv = np.real(np.fft.fft2(D1K1RealMult)) #get real part, complex part very small. 

        K10 = K0+D1K1Conv #last term is the self-energy
        
        #redefine K1 and calcualte sigma
        K1 = K10 - np.min(K10)+Delta
        
        #Check for convergence
        if(i%checkNSteps == 0):
            nStepsNow = int(i/checkNSteps)
            #calcualting sigma:
            sigma.iloc[nStepsNow] = np.array([Delta,np.dot(symTabAxis,K1.flatten())/(m*m),
                  np.dot(symTabDia,K1.flatten())/(m*m)])
            
            #check for convergence in sigma
            if(abs(sigma.iloc[nStepsNow][1] -sigma.iloc[nStepsNow-1][1])<convCrit \
                and abs(sigma.iloc[nStepsNow][2] -sigma.iloc[nStepsNow-1][2])<convCrit):
                
                stepsInN = nStepsNow
                break
            elif(i > maxNstep):
                print('sigma did not converge')
                stepsInN = nStepsNow
                break
       
    #calcualting temp from K1 and saving value.
    sigmaTemp.iloc[temp] = np.array([Delta,Temp(K1),sigma.iloc[stepsInN]['sigmaAxis'],\
                                  sigma.iloc[stepsInN]['sigmaDia']])
    

In [107]:
print(sigmaTemp.iloc[:,:2])

           Delta   Temperature
0   1.000000e-09  6.666666e-08
1   1.000000e-08  6.666663e-07
2   1.000000e-07  6.666630e-06
3   1.000000e-06  6.666298e-05
4   1.000000e-05  6.662980e-04
5   1.000000e-04  6.629978e-03
6   1.000000e-03  6.316544e-02
7   1.000000e-02  4.251742e-01
8   1.000000e-01  9.384553e-01
9   1.000000e+00  1.763336e+00
10  1.000000e+01  7.945411e+00


In [108]:
1.000000000000000062e-09 6.666666297818836472e-08 6.666666297818840442e-08
1.000000000000000021e-08 6.666662978190208671e-07 6.666662978190200201e-07
9.999999999999999547e-08 6.666629782080301772e-06 6.666629782080290760e-06
9.999999999999999547e-07 6.666297838631657726e-05 6.666297838631649595e-05
1.000000000000000082e-05 6.662980168168469155e-04 6.662980168168465903e-04
1.000000000000000048e-04 6.629978849617459633e-03 6.629978849617456163e-03
1.000000000000000021e-03 6.316550205305281052e-02 6.316550205305274113e-02
1.000000000000000021e-02 4.251760698202444178e-01 4.251760698202449729e-01
1.000000000000000056e-01 9.384553472498758930e-01 9.384553472498762261e-01
1.000000000000000000e+00 1.763335850799605087e+00 1.763335850799603310e+00
1.000000000000000000e+01 7.945410754177638424e+00 7.945410754177634871e+00


SyntaxError: invalid syntax (<ipython-input-108-feee0f070d30>, line 1)